## Building Conversational Bot using Langchain

* In this notebook we shall utilize the capabilities of langchain (PromptTemplate, LLMChain, LLM interface </br>
</br>
We shall build a conversational bot and create an interface like ChatGPT 

### Installations

In [1]:
!pip install huggingface_hub
!pip install transformers
!pip install langchain
!pip install chainlit

  Using cached chainlit-1.0.101-py3-none-any.whl (2.3 MB)
  Using cached fastapi_socketio-0.0.10-py3-none-any.whl (7.4 kB)
  Using cached Lazify-0.4.0-py2.py3-none-any.whl (3.1 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached dataclasses_json-0.5.14-py3-none-any.whl (26 kB)
  Using cached nest_asyncio-1.5.9-py3-none-any.whl (5.3 kB)
  Using cached asyncer-0.0.2-py3-none-any.whl (8.3 kB)
  Using cached literalai-0.0.102-py3-none-any.whl (30 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Attempting uninstall: nest-asyncio
    Found existing installation: nest-asyncio 1.5.1
    Uninstalling nest-asyncio-1.5.1:
      Successfully uninstalled nest-asyncio-1.5.1
  Attempting uninstall: dataclasses-json
    Found existing installation: dataclasses-json 0.6.3
    Uninstalling dataclasses-json-0.6.3:
      Successfully uninstalled dataclasses-json-0.6.3


I have already installed these libraries in my environment

In [ ]:
!chainlit hello

### Importing libraries and access token

In [ ]:
import os
import chainlit as cl
from langchain import HuggingFaceHub, PromptTemplate, LLMChain

In [ ]:
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

* The PromptTemplate is one of the elements of LangChain, necessary for building applications based on the Large Language Model. It defines how the model should interpret the user’s questions and in what context it should answer them

### Setting the conversational model

In [ ]:
#model_id = "microsoft/DialoGPT-medium" : conversational models are not currently supported by Langchain
#model_id = "mosaicml/mpt-7b-instruct"
#model_id = "tiiuae/falcon-7b"
model_id = "gpt2-medium"  #355M parameters
conv_model = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature":0.8, "max_new_tokens":200}) #0 to 1

In [ ]:
template = """You are a helpful AI assistant that makes stories by completing the query provided by the user 

{query}
"""

prompt = PromptTemplate(template=template, input_variables=['query'])

In [ ]:
conv_chain = LLMChain(llm=conv_model,
                      prompt=prompt,
                      verbose=True)

In [ ]:
print(conv_chain.run("Once upon a time in 1947"))

### Creating chatbot interface with Chainlit

Chainlit is a python package to create UI for chat interface applications </br>
We need to use the decorator from Chainlit fro langchain

https://docs.chainlit.io/overview

In [ ]:
@cl.langchain_factory(use_async=False)

def factory():
    prompt = PromptTemplate(template=template, input_variables=['question'])
    conv_chain = LLMChain(llm=conv_model,
                          prompt=prompt,
                          verbose=True)
    
    return conv_chain

### Using Conversational memory

Conversational memory is how a chatbot can respond to multiple queries in a chat-like manner </br>
It enables a coherent conversation, and without it, every query would be treated as an entirely independent input without considering past interactions

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [ ]:
user_message = "When I was a child"
while user_message!='bye':
    memory.chat_memory.add_user_message(user_message)
    resp = conv_chain.run(user_message)
    print("AI: ",resp)
    memory.chat_memory.add_ai_message(resp)
    
    user_message = input("Enter a message or bye to exit!!")

In [ ]:
template_with_memory = """You are a helpful chatbot. You answer questions 
after some thought and only provides relevant answer

Previous conversation:
{chat_history}

New human question: {question}

Response:
"""

prompt2 = PromptTemplate(template=template_with_memory, input_variables=['chat_history','question'])

In [ ]:
memory = ConversationBufferMemory(memory_key="history")

model_id = "gpt2-xl"
conv_model = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature":0.8, "max_length":128})

conv_chain_with_memory = ConversationChain(llm=conv_model,
                                  memory=memory,
                                 verbose=True)

In [ ]:
print(conv_chain_with_memory.predict(input="Hi there!"))

In [ ]:
print(conv_chain_with_memory.predict(input="Yes, I have a Mercedes. Wanna come on drive?"))

#### Types of Memory

1. ConversationBufferMemory: This memory allows for storing of messages and then extracts the messages in a 
variable
2. ConversationBufferWindowMemory: keeps a list of the interactions of the conversation over time. It only uses the last K interactions. Useful for keeping a sliding window of the most recent interactions